# NLTK

### In this part we train our machine learning algorithm to recognize positive and negative sentiments:

https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk

In [1]:
# import libraries

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
import random as random

In [2]:
# training data
data = pd.read_csv('Sentiment.csv')
data = data[['text', 'sentiment']]
data = data[data.sentiment != "Neutral"]
data.head()

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",Positive
6,RT @warriorwoman91: I liked her and was happy ...,Negative


In [3]:
# imbalanced dataset
print('Negative Sentiment count:', len(data[data['sentiment']=='Negative']))
print('Positive Sentiment count:',len(data[data['sentiment']=='Positive']))

Negative Sentiment count: 8493
Positive Sentiment count: 2236


In [4]:
# undersample
positive_df = data[data.sentiment != "Negative"]
negative_df = data[data.sentiment == "Negative"]

negative_df = negative_df[:len(positive_df)]

print('Negative Sentiment count:', len(negative_df))
print('Positive Sentiment count:', len(positive_df))

# concat the balanced data
balanced_data = pd.concat([negative_df, positive_df])
balanced_data = balanced_data.sample(frac=1).reset_index(drop=True)

balanced_data.head()

Negative Sentiment count: 2236
Positive Sentiment count: 2236


,text,sentiment
0,RT @BrianZahnd: My Native American friends get...,Negative
1,"RT @ergeekgoddess: Yo #GOPDebate, I'm really h...",Negative
2,Each #GOPdebate should be staged with one chai...,Negative
3,Babara Boxer running a CHECK on Carly Fiorina ...,Positive
4,RT @HillaryClinton: Watch the #GOPdebate? Bet ...,Negative


In [5]:
# tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')

# create a sparse matrix (BoW)
tf=TfidfVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_tf= tf.fit_transform(balanced_data['text'])

In [6]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(text_tf, balanced_data['sentiment'], test_size=0.1, random_state=123)

In [7]:
# classify

clf = MultinomialNB(alpha=0.1).fit(X_train, y_train)
predicted= clf.predict(X_test)

print("MultinomialNB Classifier Accuracy:",metrics.accuracy_score(y_test, predicted))

MultinomialNB Classifier Accuracy: 0.7901785714285714


In [8]:
# test for the first element
text_tf_2 = tf.fit_transform(balanced_data['text'])
clf.predict(text_tf_2)[0]

'Negative'

### Here, we input our own data, we clean it and we classify each tweet as positive or negative:

In [9]:
data = pd.read_csv('adhd.csv')

In [10]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

### data cleaning ###
def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

In [11]:
import re
data['clean_tweet'] = data.text.apply(clean_tweet, True)

In [12]:
tweets = data['clean_tweet']
tweets.dropna(how='all')
tweets.head()

0    saw gp today went stuff tick box adhd send ref...
1    adhd graveyard offici diagnos hobbi #adhdaware...
2    studi adult children adhd particip team sport ...
3    environment factor don’t directli caus adhd le...
4    spain celebr nation day peopl #adhd never bad ...
Name: clean_tweet, dtype: object

In [13]:
# transform the actual data
tweet_tf= tf.transform(tweets.values.astype('U'))

In [14]:
# classify
sentiment = []
for i in range(len(list(tweets))):
    s = clf.predict(tweet_tf)[i]
    sentiment.append(s)

In [15]:
# add sentiment to the dataset
data['sentiment'] = sentiment


### Sentiment Analysis done!

### Let's test a random tweet, namely 567:

In [16]:
data.text[567]

'Gender bias is leaving many #women with attention deficit hyperactivity disorder undiagnosed (#ADHD), leading psychologists are warning. BBC News https://t.co/M6GcVUa8ih'

### The algorithm outputs 'negative', which seems right:

In [17]:
data.sentiment[567]

'Negative'

### The algorithm has classified 2,777 tweets as 'Positive' and 8,025 'Negative'.

That means roughly 26% of the tweets about depression on Twitter are positive, and 74% are of negative sentiment

In [18]:
len(data[data['sentiment']=='Positive'])

2777

In [19]:
len(data[data['sentiment']=='Negative'])

8025